# 7.5 IMDb（Internet Movie Database）からDataLoaderを作成

- 本ファイルでは、IMDb（Internet Movie Database）のデータを使用して、感情分析（0：ネガティブ、1：ポジティブ）を2値クラス分類するためのDatasetとDataLoaderを作成します。


※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.5 学習目標

1.	テキスト形式のファイルデータからtsvファイルを作成し、torchtext用のDataLoaderを作成できるようになる

# 事前準備
書籍の指示に従い、本章で使用するデータを用意します


# 1. IMDbデータセットをtsv形式に変換

Datasetをダウンロードします

※torchtextで標準でIMDbが使える関数があるのですが、今回は今後データセットが用意されていない場合でも対応できるように0から作ります。

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

5万件のデータ（train,testともに2.5万件）です。データidとrating（1-10）でファイル名が決まっています。

rateは10の方が良いです。4以下がnegative、7以上がpositiveにクラス分けされています。



In [4]:
# tsv形式のファイルにします
import glob
import os
import io
import string


# 訓練データのtsvファイルを作成します

f = open('./data/IMDb_train.tsv', 'w')

path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)

path = './data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


In [5]:
# テストデータの作成

f = open('./data/IMDb_test.tsv', 'w')

path = './data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = './data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


# 2. 前処理と単語分割の関数を定義

In [6]:
import string
import re

# 以下の記号はスペースに置き換えます（カンマ、ピリオドを除く）。
# punctuationとは日本語で句点という意味です
print("区切り文字：", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 前処理


def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）


def tokenizer_punctuation(text):
    return text.strip().split()


# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 動作を確認します
print(tokenizer_with_preprocessing('I like cats.'))


区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cats', '.']


# DataLoaderの作成

In [7]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します
import torchtext


# 文章とラベルの両方に用意します
max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# 引数の意味は次の通り
# init_token：全部の文章で、文頭に入れておく単語
# eos_token：全部の文章で、文末に入れておく単語


/home/ubuntu/anaconda3/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [8]:
# フォルダ「data」から各tsvファイルを読み込みます
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 動作確認
print('訓練および検証のデータ数', len(train_val_ds))
print('1つ目の訓練および検証のデータ', vars(train_val_ds[0]))


/home/ubuntu/anaconda3/lib/python3.7/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


訓練および検証のデータ数 25000
1つ目の訓練および検証のデータ {'Text': ['i', 'just', 'saw', 'this', 'film', ',', 'i', 'first', 'saw', 'it', 'when', 'i', 'was', '7', 'and', 'could', 'just', 'about', 'remember', 'the', 'end', '.', 'so', 'i', 'watched', 'it', 'like', ',', '10', 'minutes', 'ago', ',', 'and', 'i', 'may', 'seem', 'like', 'a', 'baby', 'as', 'i', 'am', '12', 'ha', 'ha', 'i', 'started', 'to', 'cry', 'at', 'the', 'ending', ',', 'i', 'forgotten', 'how', 'sad', 'it', 'was', '.', 'i', 'think', 'i', 'was', 'mainly', 'sad', 'for', 'anne', 'marie', 'because', 'she', 'said', 'i', 'love', 'you', 'charlie', 'and', 'also', 'i', 'll', 'miss', 'you', 'charlie', ',', 'just', 'made', 'me', 'really', 'cry', 'ha', 'ha', '.', 'it', 'has', 'to', 'be', 'one', 'of', 'me', 'favourite', 'movies', 'of', 'all', 'time', ',', 'it', 'is', 'just', 'a', 'film', 'well', 'worth', 'watching', '.', 'watch', 'it', 'ha', 'ha', ',', 'thats', 'all', 'i', 'can', 'say', 'xdbut', ',', 'i', 'love', 'this', 'film', ',', 'its', 'a', 'true', 'class

In [9]:
import random
# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

# 動作確認
print('訓練データの数', len(train_ds))
print('検証データの数', len(val_ds))
print('1つ目の訓練データ', vars(train_ds[0]))


訓練データの数 20000
検証データの数 5000
1つ目の訓練データ {'Text': ['this', 'movie', 'is', 'pretty', 'awful', 'but', 'i', 'have', 'some', 'interesting', 'information', 'about', 'it', 'it', 'was', 'filmed', 'in', '1976', 'at', 'northern', 'arizona', 'university', 'in', 'flagstaff', ',', 'az', ',', 'as', 'well', 'as', 'at', 'oak', 'creek', 'canyon', 'near', 'sedona', ',', 'az', '.', 'a', 'good', 'bulk', 'of', 'the', 'extras', 'in', 'the', 'film', 'are', 'then', 'drama', 'students', 'from', 'nau', '.', 'i', 'was', 'a', 'freshman', 'there', 'that', 'year', ',', 'minoring', 'in', 'theatre', ',', 'but', 'for', 'some', 'reason', 'i', 'didn', 't', 'get', 'involved', 'with', 'the', 'production', '.', 'i', 'did', 'however', 'know', 'several', 'people', 'who', 'did', 'and', 'can', 'supply', 'this', 'rather', 'odd', 'fact', 'there', 'is', 'a', 'scene', 'in', 'this', 'movie', 'where', 'two', 'of', 'the', 'principals', ',', 'as', 'part', 'of', 'their', 'hazing', 'ritual', ',', 'have', 'to', 'run', 'naked', 'into', 'the'

# ボキャブラリーを作成

In [10]:
# torchtextで単語ベクトルとして英語学習済みモデルを読み込みます

from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')


# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", english_fasttext_vectors.dim)
print("単語数：", len(english_fasttext_vectors.itos))


100%|█████████▉| 999939/999994 [02:20<00:00, 6430.20it/s]

1単語を表現する次元数： 300
単語数： 999994


100%|█████████▉| 999939/999994 [02:30<00:00, 6430.20it/s]

In [11]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 17916個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


torch.Size([17903, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7faa30344510>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             'the': 4,
             '.': 5,
             ',': 6,
             'and': 7,
             'a': 8,
             'of': 9,
             'to': 10,
             'is': 11,
             'it': 12,
             'in': 13,
             'i': 14,
             'this': 15,
             'that': 16,
             's': 17,
             'was': 18,
             'as': 19,
             'for': 20,
             'movie': 21,
             'with': 22,
             'but': 23,
             'film': 24,
             'on': 25,
             't': 26,
             'you': 27,
             'not': 28,
             'he': 29,
             'his': 30,
             'are': 31,
             'have': 32,
             'be': 33,
             'one': 34,
             'all': 35,
             'at': 36,
             'they': 37,

In [12]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=24, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=24, train=False, sort=False)


# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


(tensor([[   2,   76,    6,  ...,    1,    1,    1],
        [   2,   15,   11,  ...,    1,    1,    1],
        [   2,   14,  309,  ...,    1,    1,    1],
        ...,
        [   2,  887,  393,  ..., 3815,    4,    3],
        [   2, 1293,  911,  ...,    9, 2671,    3],
        [   2,    0, 2935,  ...,    9,  653,    3]]), tensor([115, 150, 194, 210, 186, 126, 209, 124, 205, 145, 256, 160, 135,  68,
        118, 187, 240, 197, 256, 256, 134, 256, 256, 256]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])


/home/ubuntu/anaconda3/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
